In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import glob
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
from statistics import mean
from random import choice
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn import preprocessing
from gendis.genetic import GeneticExtractor
from tools.utils import scale_down, standardize_data, set_movements, angle_diff, get_distance
train_test_options = {"split": 25}


    
def read_ship_csv(file):
    ship =  pd.read_csv(file)
    shipname = ship.loc[0]["SHIPNAME"]
    ship = ship [["TIMESTAMP","LAT","LON","HEADING"]]
    ship['TIMESTAMP'] = pd.to_datetime(ship['TIMESTAMP'])  
    ship.sort_values('TIMESTAMP',inplace=True)
    ship=ship.reset_index(drop=True)
    return ship,shipname

In [ ]:
if len(glob.glob('ships')) != 0 :
    files = glob.glob("ships/*.csv")
    all_ships = [] 
    for file in files:
        ship,shipname=read_ship_csv(file)
        n = 500  #chunk row size
        ship_dfs = [ship[i:i+n] for i in range(0,ship.shape[0],n)]
        ship_data_chunked = []
        ship_data_chunked_index = [False for i in range(0,len(ship_dfs))]  
        for idx,x in enumerate(ship_dfs):
            x = scale_down(x,train_test_options["split"])
            x = np.array(x["HEADING"].values.astype(int))
            if len(x) == train_test_options["split"]:
                ship_data_chunked_index[idx] = True
                ship_data_chunked.append(x)
        ship_data_chunked = np.array(ship_data_chunked)
        ship_dfs = [ship for (ship,index) in zip(ship_dfs,ship_data_chunked_index) if  index ]
        ship ={"shipname":shipname,"data":ship_data_chunked,"ship_dfs":ship_dfs}
        print("ERROR DIFFERENT SIZE OF DATA") if len(ship_dfs)!=ship["data"].shape[0] else None
        all_ships.append(ship)


In [ ]:
for ship in all_ships:
    os.mkdir("figures/zoomed_plots/"+ship["shipname"])
    for idx,x in enumerate(ship["ship_dfs"]):
        df_list = []
        data_list = []

        data_scaled=scale_down(x,train_test_options["split"]).assign(Trajectory="Scaled Trajectory")
        df_list.append(data_scaled)
        data_list.append(data_scaled["Trajectory"].iloc[0])
        pallete=['b']
        sizes=[3]
        dashes=[""]
        markers=["o"]

        start = data_scaled.head(1)["TIMESTAMP"].values
        end = data_scaled.tail(1)["TIMESTAMP"].values
        date_mask = (x["TIMESTAMP"].values>=start) & (x["TIMESTAMP"].values<=end)
        data=x[date_mask]

        real_data=data.assign(Trajectory="Real Trajectory")
        df_list.append(real_data)
        data_list.append(real_data["Trajectory"].iloc[0])
        pallete.append('r')
        markers = ["^"]
        sizes.append(6)
        dashes.append((5,5))
        markers.append("^")

        merged_data = pd.concat(df_list)

        fig, ax= plt.subplots(1, 1, figsize=(50, 30),sharex=True)
        start = datetime.strftime(data["TIMESTAMP"].head(1).iloc[0],'%Y-%m-%d %H:%M:%S')
        end = datetime.strftime(data["TIMESTAMP"].tail(1).iloc[0],'%Y-%m-%d %H:%M:%S')
        title = "Vessel:"+ship["shipname"]+"\nCHUNK NO: "+str(idx)+"\n"+start+"----"+end
        ax.set_title(title,fontsize=50)
        sns.lineplot(x='LON', y='LAT',
                          data=merged_data,
                          ax=ax,
                          sort=False,
                          hue = "Trajectory",
                          palette = pallete,
                          hue_order= data_list,
                          size=merged_data["Trajectory"],
                          sizes = sizes,
                          size_order = data_list,
                      style = merged_data["Trajectory"],
                      style_order = data_list,
                      dashes = dashes,
                      markers = markers,
                      legend ="full",
                     )
        lgnd = plt.legend(loc="upper center", numpoints=1, fontsize=25)
        ax.set_ylabel('Latitute', fontsize = 20.0) # Y label
        ax.set_xlabel('Longitude ', fontsize = 20.0) # X label

        plt.savefig("figures/zoomed_plots/"+ship["shipname"]+"/chunk_no_"+str(idx)+".png",dpi=fig.dpi)
        plt.clf()
